# 🎯 Face Detection and Alignment with UniFace

<div style="display:flex; flex-wrap:wrap; align-items:center;">
  <a style="margin-right:10px; margin-bottom:6px;" href="https://pepy.tech/projects/uniface"><img alt="PyPI Downloads" src="https://static.pepy.tech/badge/uniface"></a>
  <a style="margin-right:10px; margin-bottom:6px;" href="https://pypi.org/project/uniface/"><img alt="PyPI Version" src="https://img.shields.io/pypi/v/uniface.svg"></a>
  <a style="margin-right:10px; margin-bottom:6px;" href="https://opensource.org/licenses/MIT"><img alt="License" src="https://img.shields.io/badge/License-MIT-blue.svg"></a>
  <a style="margin-bottom:6px;" href="https://github.com/yakhyo/uniface"><img alt="GitHub Stars" src="https://img.shields.io/github/stars/yakhyo/uniface.svg?style=social"></a>
</div>

**UniFace** is a lightweight, production-ready, all-in-one face analysis library built on ONNX Runtime.

🔗 **GitHub**: [github.com/yakhyo/uniface](https://github.com/yakhyo/uniface) | 📚 **Docs**: [yakhyo.github.io/uniface](https://yakhyo.github.io/uniface)

---

## 📖 Overview

This notebook demonstrates **face alignment** - a crucial preprocessing step for face recognition:

- ✅ Detect faces and extract 5-point landmarks
- ✅ Align faces using similarity transformation
- ✅ Generate normalized 112×112 face crops ready for recognition

## 1️⃣ Installation

In [ ]:
%pip install -q uniface

import os
import urllib.request

os.makedirs('assets/test_images', exist_ok=True)

BASE_URL = "https://raw.githubusercontent.com/yakhyo/uniface/main/assets"
images = ["test_images/image0.jpg", "test_images/image1.jpg", "test_images/image2.jpg",
          "test_images/image3.jpg", "test_images/image4.jpg"]

for img in images:
    if not os.path.exists(f'assets/{img}'):
        urllib.request.urlretrieve(f"{BASE_URL}/{img}", f"assets/{img}")
        print(f"✓ Downloaded {img}")

## 2️⃣ Import Libraries

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

import uniface
from uniface.detection import RetinaFace
from uniface.face_utils import face_alignment
from uniface.visualization import draw_detections

print(f"UniFace version: {uniface.__version__}")

## 3️⃣ Initialize the Detector

In [ ]:
detector = RetinaFace(
    confidence_threshold=0.5,
    nms_threshold=0.4,
)

## 4️⃣ Load Images and Perform Detection + Alignment

We'll process multiple images to demonstrate the alignment pipeline:
1. **Detect** faces and extract landmarks
2. **Align** faces using 5-point landmarks
3. **Crop** to 112×112 (standard size for face recognition)

In [ ]:
image_paths = [
    'assets/test_images/image0.jpg',
    'assets/test_images/image1.jpg',
    'assets/test_images/image2.jpg',
    'assets/test_images/image3.jpg',
    'assets/test_images/image4.jpg',
]

original_images = []
detection_images = []
aligned_images = []

for image_path in image_paths:
    image = cv2.imread(image_path)
    if image is None:
        print(f'⚠ Error: Could not read {image_path}')
        continue

    faces = detector.detect(image)
    if not faces:
        print(f'⚠ No faces detected in {image_path}')
        continue

    bbox_image = image.copy()
    bboxes = [f.bbox for f in faces]
    scores = [f.confidence for f in faces]
    landmarks = [f.landmarks for f in faces]
    draw_detections(image=bbox_image, bboxes=bboxes, scores=scores, landmarks=landmarks, vis_threshold=0.6, fancy_bbox=True)

    first_landmarks = faces[0].landmarks
    aligned_image, _ = face_alignment(image, first_landmarks, image_size=112)

    original_images.append(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    detection_images.append(cv2.cvtColor(bbox_image, cv2.COLOR_BGR2RGB))
    aligned_images.append(cv2.cvtColor(aligned_image, cv2.COLOR_BGR2RGB))

print(f'✓ Processed {len(original_images)} images')

## 5️⃣ Visualize Results

**Row 1**: Original images  
**Row 2**: Detection with landmarks  
**Row 3**: Aligned 112×112 face crops

In [ ]:
fig, axes = plt.subplots(3, len(original_images), figsize=(15, 10))

row_titles = ['Original', 'Detection', 'Aligned (112×112)']

for row, images in enumerate([original_images, detection_images, aligned_images]):
    for col, img in enumerate(images):
        axes[row, col].imshow(img)
        axes[row, col].axis('off')
        if col == 0:
            axes[row, col].set_title(row_titles[row], fontsize=12, loc='left')

plt.tight_layout()
plt.show()

---

## 📝 Summary

| Feature | Description |
|---------|-------------|
| **Detection** | `detect()` returns `Face` objects with `bbox`, `confidence`, `landmarks` |
| **Landmarks** | 5-point facial landmarks (2 eyes, nose, 2 mouth corners) |
| **Alignment** | `face_alignment()` uses similarity transform to normalize faces |
| **Output Size** | Default 112×112 (standard for recognition models) |

### Why Face Alignment Matters

1. **Consistency**: Normalized face positions improve recognition accuracy
2. **Scale invariance**: All faces are scaled to the same size
3. **Rotation correction**: Faces are rotated to a canonical orientation

---

## 🔗 Next Steps

- **Face Verification**: Compare two faces → [03_face_verification.ipynb](./03_face_verification.ipynb)
- **Face Search**: Find a person in a group → [04_face_search.ipynb](./04_face_search.ipynb)
- **Full Documentation**: [yakhyo.github.io/uniface](https://yakhyo.github.io/uniface)